In [200]:
import datetime
import glob
import os

from arelle.ModelValue import qname
from arelle import Cntlr
import numpy as np

import pandas as pd
pd.reset_option('display.max_rows')
print(pd.options.display.max_rows)
pd.reset_option('display.max_columns')
print(pd.options.display.max_columns)
import plotly.graph_objects as go

import stock_research as sr
taiouhyou = sr.taiouhyou
import warnings
warnings.filterwarnings('ignore')

60
20


In [201]:
result_path = '../warehouse/stock/created/result_statistics/*'
files = glob.glob(result_path)
for file in files:
    print(file)
result_path = '../warehouse/stock/created/result_statistics\stock_statistics_20170301_20211201.csv'

../warehouse/stock/created/result_statistics\report.txt
../warehouse/stock/created/result_statistics\stock_statistics_20170301_20211201.csv


In [202]:
result = pd.read_csv(result_path)

In [203]:
today = datetime.datetime.today().strftime('%Y%m%d')
filepath = f'../warehouse/stock/created/result_statistics/{today}_report.txt'
th_opemargin = 40
th_sales_CAGR = 1.30
sentence = f'報告日 {today} 基準値\n営業CFマージン {th_opemargin}%\tCAGR {int(th_sales_CAGR*100)}%'+'\n'
sentence = today + sentence

In [208]:
with open(filepath, mode='a') as f:
    f.write(sentence)

res_dict = {}
for stock in result.secCode.unique():
    res_dict[f'{stock}'] = {}
    sdf = result.query(f'secCode=={stock}')
    filerName = sdf.filerName.reset_index(drop=True)[0]
    
    
    # 足切り 純利益が赤字
    
    # 営業キャッシュフローマージンが平均40%以上
    mean_1 = sdf.operatingincome_margin.replace('%','',regex=True).astype(float).mean()
    count_1 = sdf.operatingincome_margin.replace('%','',regex=True).astype(float).count()
    
    if mean_1 > th_opemargin:
        s = f'''{stock} {filerName} {count_1}年 {mean_1:.2f}% > {th_opemargin}%\n'''
        res_dict[f'{stock}']['opeCF']=[count_1,mean_1]
        with open(filepath, mode='a') as f:
            f.write(s)

    #############################################################
    # 連結会計
    tmp = sdf.query('NonConsolidated==0')
    tmp = tmp.query('element_id=="NetSalesSummaryOfBusinessResults"')
    
    if len(tmp)<2:
        continue
    
    # 幾何平均
    gmean_2 = (((tmp.Yoy.replace('%','',regex=True).astype(float)/100)[1:].cumprod(axis=0)[-1:]**(1/len((tmp.Yoy)[1:]))).values[0])
    count_2 = tmp.Yoy.replace('%','',regex=True).astype(float).count()
    if gmean_ > th_sales_CAGR:
        s = f'''{stock} {filerName} CAGR({count_2}) {gmean_2:.3f}% > {threshold}%\n'''
        # print(s)
        res_dict[f'{stock}']['CAGR']=[count_2,gmean_2]
        with open(filepath, mode='a') as f:
            f.write(s)
    #############################################################
    
    if (gmean_ > th_sales_CAGR) and (mean_1 > th_opemargin):
        print(f'''{stock} {filerName} CAGR({count_2}) > {gmean_2:.3f}%\n営業CFマージン{th_opemargin} {count_1}年 {mean_1:.2f}% > {threshold}%\n''')
    
    

In [207]:
result

,element_id,value,contextID,startDatetime,endDatetime,instantDatetime,NonConsolidated,value_sales,operatingincome_margin,filerName,secCode,33業種区分,17業種区分,lag,Yoy
0,NetSalesSummaryOfBusinessResults,2.381937e+10,Prior4YearDuration_NonConsolidatedMember,2016-10-01,2017-10-01,NaN,1,2.381937e+10,NaN,太洋物産株式会社,9941,卸売業,商社・卸売,NaN,NaN
1,NetSalesSummaryOfBusinessResults,2.005576e+10,Prior3YearDuration_NonConsolidatedMember,2017-10-01,2018-10-01,NaN,1,2.005576e+10,71%,太洋物産株式会社,9941,卸売業,商社・卸売,2.381937e+10,84%
2,NetSalesSummaryOfBusinessResults,1.951931e+10,Prior2YearDuration_NonConsolidatedMember,2018-10-01,2019-10-01,NaN,1,1.951931e+10,32%,太洋物産株式会社,9941,卸売業,商社・卸売,2.005576e+10,97%
3,NetSalesSummaryOfBusinessResults,1.480093e+10,Prior1YearDuration_NonConsolidatedMember,2019-10-01,2020-10-01,NaN,1,1.480093e+10,NaN,太洋物産株式会社,9941,卸売業,商社・卸売,1.951931e+10,75%
4,NetSalesSummaryOfBusinessResults,1.642366e+10,CurrentYearDuration_NonConsolidatedMember,2020-10-01,2021-10-01,NaN,1,1.642366e+10,39%,太洋物産株式会社,9941,卸売業,商社・卸売,1.480093e+10,110%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1517,NetSalesSummaryOfBusinessResults,1.850900e+10,Prior1YearDuration_NonConsolidatedMember,2019-12-01,2020-12-01,NaN,1,1.850900e+10,NaN,津田駒工業株式会社,6217,機械,機械,3.333500e+10,55%
1518,NetSalesSummaryOfBusinessResults,2.392200e+10,CurrentYearDuration_NonConsolidatedMember,2020-12-01,2021-12-01,NaN,1,2.392200e+10,NaN,津田駒工業株式会社,6217,機械,機械,1.850900e+10,129%
1519,NetCashProvidedByUsedInOperatingActivitiesSumm...,4.080000e+08,Prior4YearDuration,2016-12-01,2017-12-01,NaN,0,NaN,NaN,津田駒工業株式会社,6217,機械,機械,NaN,NaN
1520,NetCashProvidedByUsedInOperatingActivitiesSumm...,1.983000e+09,Prior3YearDuration,2017-12-01,2018-12-01,NaN,0,NaN,NaN,津田駒工業株式会社,6217,機械,機械,4.080000e+08,486%
